In [22]:
# Manual Selection of Trackings
CLASSES = "RURAL", "SUBURBAN", "TOWN", "URBAN"

trackings = {
    "8aa43ce03bc54486875ef97f680c06e6": "RURAL", 
    "98903798caa1452d94905904c490194a": "RURAL", 
    "bf6e2ac6945c4091ae15c855c2beca05": "TOWN",
    "f7cf22a205cd41bba9308d38ff956bb4": "TOWN",
    "2559726e33be48fdaee002f3dec7251e": "SUBURBAN",
    "6e1f92dc3e734b5eb965a343781be443": "SUBURBAN",
    "c535c3f521524ffc873b9e89a57b492e": "URBAN", # Linz
    "0975abe81a084daeaf8b1b03f9afd981": "URBAN" # Linz
}

In [27]:
from sqlalchemy import create_engine
import pandas as pd
import json

with open('../../../Credentials/awm_database_credentials.json') as data_file:
    data = json.load(data_file)
    
HOST = data['host']
PORT = data['port']
USER = data['user']
PASSWORD = data['password']
DATABASE = data['database']

engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

In [30]:
tracking_names_sql = ",".join(f"'{name}'" for name in trackings.keys())

query = f"""
SELECT id, name
FROM tracking
WHERE name IN ({tracking_names_sql})
"""

tracking_df = pd.read_sql(query, engine)

tracking_df["label"] = tracking_df["name"].map(trackings)

tracking_df.to_json("../Sample_Data/tracking_ids_and_labels.json", orient="records", indent=2)

print(tracking_df)

       id                              name     label
0   99476  0975abe81a084daeaf8b1b03f9afd981     URBAN
1   99713  2559726e33be48fdaee002f3dec7251e  SUBURBAN
2   99307  6e1f92dc3e734b5eb965a343781be443  SUBURBAN
3  100913  8aa43ce03bc54486875ef97f680c06e6     RURAL
4   99944  98903798caa1452d94905904c490194a     RURAL
5  100012  bf6e2ac6945c4091ae15c855c2beca05      TOWN
6   99496  c535c3f521524ffc873b9e89a57b492e     URBAN
7   99729  f7cf22a205cd41bba9308d38ff956bb4      TOWN


In [ ]:
import json
import asyncio
from AWM_API.api_calls import get_access_token, get_single_tracking

with open("../Sample_Data/tracking_ids_and_labels.json", "r", encoding="utf-8") as f:
    tracking_info = json.load(f)

async def fetch_and_save_from_ids():
    access_token = await get_access_token()
    if not access_token:
        print("Failed to get access token.")
        return

    labeled_data = []

    for tracking in tracking_info:
        tracking_id = tracking["id"]
        label = tracking["label"]
        print(f"Fetching tracking {tracking_id} ({label})...")

        data = await get_single_tracking(access_token, str(tracking_id))
        if data:
            data["label"] = label
            labeled_data.append(data)

    with open("../Sample_Data/labeled_trackings_from_api.json", "w", encoding="utf-8") as f:
        json.dump(labeled_data, f, indent=2, ensure_ascii=False)

    print("Labeled trackings saved to labeled_trackings_from_api.json")

await fetch_and_save_from_ids()  



<Response [200 OK]>
Fetching tracking 99476 (URBAN)...
Fetching tracking 99713 (SUBURBAN)...
Fetching tracking 99307 (SUBURBAN)...
Fetching tracking 100913 (RURAL)...
Fetching tracking 99944 (RURAL)...
Fetching tracking 100012 (TOWN)...
Fetching tracking 99496 (URBAN)...
Fetching tracking 99729 (TOWN)...
✅ Labeled trackings saved to labeled_trackings_from_api.json
